In [6]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,LeakyReLU
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [7]:
class DNN(Model):
    def __init__(self,hidden_dim,output_dim):
        super().__init__()
        self.l1=Dense(hidden_dim)
        self.a1=LeakyReLU(0.01)
        self.l2=Dense(hidden_dim)
        self.a2=LeakyReLU(0.01)
        self.l3=Dense(hidden_dim)
        self.a3=LeakyReLU(0.01)
        self.l4=Dense(output_dim,activation='softmax')
        self.ls=[self.l1,self.a1,self.l2,self.a2,self.l3,self.a3,self.l4]
        
    def call(self,x):
        for layer in self.ls:
            x=layer(x)
        return x

In [8]:
mnist=datasets.mnist
(x_train,t_train),(x_test,t_test)=mnist.load_data()

x_train=(x_train.reshape(-1,784)/255).astype(np.float32)
x_test=(x_test.reshape(-1,784)/255).astype(np.float32)
t_train=np.eye(10)[t_train].astype(np.float32)
t_test=np.eye(10)[t_test].astype(np.float32)

In [9]:
model=DNN(200,10)

In [10]:
criterion=losses.CategoricalCrossentropy()
optimizer=optimizers.SGD(learning_rate=0.1)
train_loss=metrics.Mean()
train_acc=metrics.CategoricalAccuracy()

def compute_loss(t,y):
    return criterion(t,y)

def train_step(x,t):
    with tf.GradientTape() as tape:
        preds=model(x)
        loss=compute_loss(preds,t)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss)
    train_acc(t,preds)
    return loss

epochs=30
batch_size=100
n_batches=x_train.shape[0]//batch_size

for epoch in range(epochs):
    x_,t_=shuffle(x_train,t_train)
    
    for batch in range(n_batches):
        start=batch*batch_size
        end=start+batch_size
        train_step(x_[start:end],t_[start:end])
    
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}'.format(epoch,train_loss.result(),train_acc.result()))


epoch: 0, loss: 3.119, acc: 0.813
epoch: 1, loss: 2.169, acc: 0.869
epoch: 2, loss: 1.834, acc: 0.889
epoch: 3, loss: 1.668, acc: 0.898
epoch: 4, loss: 1.587, acc: 0.903
epoch: 5, loss: 1.554, acc: 0.905
epoch: 6, loss: 1.552, acc: 0.905
epoch: 7, loss: 1.724, acc: 0.894
epoch: 8, loss: 2.019, acc: 0.876
epoch: 9, loss: 2.405, acc: 0.852
epoch: 10, loss: 3.135, acc: 0.806
epoch: 11, loss: 3.817, acc: 0.764
epoch: 12, loss: 4.400, acc: 0.728
epoch: 13, loss: 4.914, acc: 0.696
epoch: 14, loss: 5.416, acc: 0.664
epoch: 15, loss: 5.853, acc: 0.637
epoch: 16, loss: 6.333, acc: 0.608
epoch: 17, loss: 6.788, acc: 0.579
epoch: 18, loss: 7.195, acc: 0.554
epoch: 19, loss: 7.562, acc: 0.531
epoch: 20, loss: 7.894, acc: 0.511
epoch: 21, loss: 8.195, acc: 0.492
epoch: 22, loss: 8.471, acc: 0.475
epoch: 23, loss: 8.723, acc: 0.459
epoch: 24, loss: 8.955, acc: 0.445
epoch: 25, loss: 9.170, acc: 0.431
epoch: 26, loss: 9.368, acc: 0.419
epoch: 27, loss: 9.552, acc: 0.408
epoch: 28, loss: 9.724, acc: 0

In [11]:
test_loss=metrics.Mean()
test_acc=metrics.CategoricalAccuracy()

def test_step(x,t):
    preds=model(x)
    loss=compute_loss(t,preds)
    test_loss(loss)
    test_acc(t,preds)
    return loss

test_step(x_test,t_test)

print('test_loss: {:.3f}, test_acc:{:.3f}'.format(test_loss.result(),test_acc.result()))

test_loss: 14.574, test_acc:0.096
